In [126]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame,Series
import os
data_path = 'output'
field_to_count='id' # Field that contains a value (not null) in each row.
percentage = 'percentage (%)'
count = 'count'
time_data = ['hour','day']
from pandas.api.types import CategoricalDtype
cats = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
cat_type = CategoricalDtype(categories=cats, ordered=True)


1

In [39]:
path = "wrm_demographics_2022-12-05.csv"
ir_fields =['description','geometry',]
data_0 = pd.read_csv(path)
data = data_0.drop(ir_fields,axis=1)
data

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,mobility_aid_type,feature_type,feature_subtype,lon,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class
0,4,2021-05-27T22:34:00Z,2021-05-28T22:50:36Z,incident,1971,female,white,no,NaN,no response,NaN,near-miss,vehicle-turning-head-on,-123.434508,48.459019,Women,50,45-54,No response,None
1,7,2021-05-29T19:46:42Z,2021-05-29T20:01:09Z,incident,2000,no-response,no-response,no,NaN,no response,NaN,near-miss,vehicle-turning-right,-123.316472,48.433674,No Response,21,18-24,No response,None
2,8,2021-05-31T17:54:58Z,2021-05-31T18:00:16Z,amenity,1982,male,white,no,NaN,no response,NaN,sidewalk,NaN,-123.039638,49.288649,Men,39,35-44,No response,None
3,16,2021-06-04T17:31:46Z,2021-06-04T17:56:33Z,incident,1982,male,white,no,NaN,no response,NaN,near-miss,vehicle-from-behind,-123.036018,49.281230,Men,39,35-44,No response,None
4,17,2021-06-04T21:35:00Z,2021-06-04T21:41:11Z,amenity,1993,female,white,no,NaN,no response,NaN,transit,NaN,-123.193901,49.263620,Women,28,25-34,No response,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1987,2022-11-27T18:00:00Z,2022-12-04T20:03:10Z,hazard-concern,1975,male,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.700354,45.497378,Men,47,45-54,No response,None
1384,1988,2022-12-04T19:55:30Z,2022-12-04T20:05:26Z,hazard-concern,1975,female,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.699729,45.501452,Women,47,45-54,No response,None
1385,1989,2022-12-04T19:55:30Z,2022-12-04T20:08:21Z,amenity,1986,male,white,no,NaN,no response,NaN,sidewalk,NaN,-75.696871,45.496891,Men,36,35-44,No response,None
1386,1990,2022-12-04T19:55:30Z,2022-12-04T20:11:31Z,amenity,1962,male,latino,no,NaN,no response,NaN,sidewalk,NaN,-75.691793,45.504402,Men,60,55-64,No response,None


In [47]:
data.columns

Index(['id', 'date', 'date_reported', 'type', 'birth_year', 'gender', 'race',
       'disability', 'disability_type', 'mobility_aid', 'mobility_aid_type',
       'feature_type', 'feature_subtype', 'lon', 'lat', 'gender_class', 'age',
       'age_class', 'mobility_aid_type_class', 'disability_type_class', 'hour',
       'day'],
      dtype='object')

In [56]:
for i in range(1,len(data.columns)):
    print(DataFrame(data.groupby(data.columns[i]).count()['id'].rename('count').sort_values(ascending=False)))
    print('\n')

                      count
date                       
2022-03-01T16:06:54Z     19
2022-11-02T16:36:45Z     11
2022-11-09T20:02:47Z     10
2022-02-01T19:05:13Z      8
2022-05-11T17:08:24Z      8
...                     ...
2022-02-01T16:06:00Z      1
2022-02-01T16:09:17Z      1
2022-02-01T16:49:59Z      1
2022-02-01T18:17:17Z      1
2022-04-15T17:50:00Z      1

[1084 rows x 1 columns]


                      count
date_reported              
2022-05-11T18:17:20Z      3
2021-12-08T16:20:36Z      3
2022-08-21T19:14:12Z      2
2022-01-10T03:12:37Z      2
2022-08-11T03:27:50Z      1
...                     ...
2022-02-24T17:18:05Z      1
2022-02-24T17:08:02Z      1
2022-02-24T16:56:41Z      1
2022-02-24T16:50:43Z      1
2022-12-04T20:13:56Z      1

[1382 rows x 1 columns]


                count
type                 
hazard-concern    737
amenity           475
incident          176


            count
birth_year       
1986          102
1997           51
1985           50
1971           4

### Univariate variables

In [110]:
univariate_columns =['date','type','gender_class','race','disability','disability_type','mobility_aid','feature_type','feature_subtype','age_class', 'mobility_aid_type_class', 'disability_type_class']

In [112]:
for i in univariate_columns:
    print(DataFrame(data.groupby(data[i]).count()['id'].rename('count').sort_values(ascending=False)))
    print('\n')

                      count
date                       
2022-03-01T16:06:54Z     19
2022-11-02T16:36:45Z     11
2022-11-09T20:02:47Z     10
2022-02-01T19:05:13Z      8
2022-05-11T17:08:24Z      8
...                     ...
2022-02-01T16:06:00Z      1
2022-02-01T16:09:17Z      1
2022-02-01T16:49:59Z      1
2022-02-01T18:17:17Z      1
2022-04-15T17:50:00Z      1

[1084 rows x 1 columns]


                count
type                 
hazard-concern    737
amenity           475
incident          176


              count
gender_class       
Women           670
Men             522
No Response     174
Non-Binary       22


                                          count
race                                           
white                                      1009
no response                                 233
other                                        18
indigenous                                   16
southeast asian                              13
south asisan                           

#### Time (time and weekend VS week)

In [131]:
def var_groupby(groupby_field:str)-> DataFrame:
    """
    The method counts the number of results (and percentage) in the same category.
    :param groupby_field
    :return:
    """
    groupby_data = DataFrame(data.groupby(data[groupby_field]).count()[field_to_count].rename(count))
    groupby_data [percentage] = round(groupby_data[count]/groupby_data[count].sum()*100,1)
    return groupby_data

NameError: name 'cat_type' is not defined

In [120]:

new_tables = {}

for var in univariate_columns:
    # calculate group by and percentage

    if var=='date':
        # specific analysis
        date = pd.to_datetime((data['date'].str.replace('Z','').str.replace('T',' ')))
        hour,day  = 'hour','day'
        data[hour],data[day] = date.dt.hour, date.dt.day_name()
        #  sort by weekday
        new_tables[hour] = var_groupby(groupby_field=hour)
        new_tables[day][day]  =  var_groupby(groupby_field=day).astype(cat_type)
    else:
        new_tables[var] = var_groupby(groupby_field=var).sort_values(by=percentage,ascending=False)

# save
with pd.ExcelWriter(f"{data_path}/univariate.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [40]:
date = pd.to_datetime((data['date'].str.replace('Z','').str.replace('T',' ')))

### Incidents per hour

In [ ]:
new_tables


In [123]:
data[data['hour']==10]

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,...,feature_subtype,lon,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class,hour,day
700,1219,2022-05-01T10:50:00Z,2022-05-08T18:37:07Z,amenity,1986,female,white,no,NaN,no response,...,NaN,-123.323447,48.410341,Women,36,35-44,No response,None,10,Sunday
810,1367,2022-06-29T10:09:04Z,2022-06-29T10:13:49Z,amenity,1977,female,white,no,NaN,no response,...,NaN,-82.292687,34.892758,Women,45,45-54,No response,None,10,Wednesday
848,1405,2022-07-12T10:56:17Z,2022-07-12T20:44:59Z,hazard-concern,1950,male,white,no response,NaN,no response,...,"obstruction - uneven sidewalk (eg. roots, hole...",-75.687187,45.343528,Men,72,65-74,No response,No Response,10,Tuesday


In [105]:
data['hour'] = date.dt.hour
incidents_per_hour = DataFrame(data.groupby(data['hour']).count()['id'].rename('count'))
incidents_per_hour['percentage'] = round(incidents_per_hour['count']/incidents_per_hour['count'].sum()*100,1)
incidents_per_hour

,count,percentage
hour,,
0,69,5.0
1,84,6.1
2,45,3.2
3,45,3.2
4,34,2.4
5,27,1.9
6,7,0.5
7,11,0.8
8,4,0.3


### Incidents per day (Monday=0, Sunday=6)

In [113]:
data['day'] = date.dt.day_name()


0      2021-05-27 22:34:00
1      2021-05-29 19:46:42
2      2021-05-31 17:54:58
3      2021-06-04 17:31:46
4      2021-06-04 21:35:00
               ...        
1383   2022-11-27 18:00:00
1384   2022-12-04 19:55:30
1385   2022-12-04 19:55:30
1386   2022-12-04 19:55:30
1387   2022-12-04 19:55:30
Name: date, Length: 1388, dtype: datetime64[ns]

In [114]:
data['day']

0       Thursday
1       Saturday
2         Monday
3         Friday
4         Friday
          ...   
1383      Sunday
1384      Sunday
1385      Sunday
1386      Sunday
1387      Sunday
Name: day, Length: 1388, dtype: object

In [ ]:
incidents_per_day= DataFrame(data.groupby(data['day']).count()['id'].rename('count'))
incidents_per_hour['percentage'] = round(incidents_per_hour['count']/incidents_per_hour['count'].sum()*100,1)
incidents_per_day

In [ ]:
with pd.ExcelWriter(f"{data_path }/output.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [95]:
def bivariate_analysis(main_vars:list, secondary_vars:list, name_analysis:str, field_to_store:str, staty:dict):
    """
    It goes over the dictionary and calculate the cross relation between the variable inside
    :param staty: contain summery data of the main vars
    :param field_to_store: which field to retain after groupby and counting
    :param name_analysis: it is used to create new folder if necessary
    :param main_vars:
    :param secondary_vars:
    :return:
    """

    res_path = f'{data_path}/{name_analysis}'
    os.makedirs(res_path ,exist_ok = True)
    for main_var in main_vars:
        print(f'\n{main_var}')
        new_tables_data = {}
        for other in secondary_vars:
            print(f'{other}\t',end="")
            # GroupBy two questions (one of which is main_var)
            table_00 = DataFrame(data.groupby([main_var, other]).count()[field_to_store].rename(''))
            # Get the data in readable structure
            table_0 =table_00.unstack(fill_value=0)
            # Clean columns and index
            table_0.columns = table_0.columns.droplevel()

            # get percentage (questions variables for each group)
            old_columns =table_0.columns
            table_2 = table_0.copy()
            sum_field = staty[main_var]['count']
            table_2[old_columns +' (%)'] = round(table_0.div(sum_field, axis=0) * 100, 1)
            table_2['sum_field'] = sum_field
            new_tables_data[f'{other}'] =table_2
        with pd.ExcelWriter(f"{res_path}/{main_var}.xlsx") as writer:
            [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]

In [96]:


stat= {time_data[0]:DataFrame(incidents_per_hour),time_data[1]:DataFrame(incidents_per_day)}
other_data = ['type', 'feature_type', 'feature_subtype']
bivariate_analysis(main_vars=time_data, secondary_vars=other_data, name_analysis='time', field_to_store='id', staty=stat)


hour
type	feature_type	feature_subtype	
day
type	feature_type	feature_subtype	

In [92]:
main_vars=time_data
secondary_vars = other_data
name_analysis='time'
field_to_store='id'
staty=stat



res_path = f'{data_path}/{name_analysis}'
os.makedirs(res_path ,exist_ok = True)
for main_var in main_vars:
    print(main_var)
    new_tables_data = {}
    for other in secondary_vars:
        print(f'{other}\t',end="")
        # GroupBy two questions (one of which is main_var)
        table_00 = DataFrame(data.groupby([main_var, other]).count()[field_to_store].rename(''))
        # Get the data in readable structure
        table_0 =table_00.unstack(fill_value=0)
        # Clean columns and index
        table_0.columns = table_0.columns.droplevel()

        # get percentage (questions variables for each group)
        old_columns =table_0.columns
        table_2 = table_0.copy()
        sum_field = staty[main_var]['count']
        table_2[old_columns +' (%)'] = round(table_0.div(sum_field, axis=0) * 100, 1)
        table_2['sum_field'] = sum_field
        new_tables_data[f'{other}'] =table_2
    with pd.ExcelWriter(f"{res_path}/{main_var}.xlsx") as writer:
        [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]

hour
type	feature_type	feature_subtype	day
type	feature_type	feature_subtype	

In [91]:
table_2

feature_subtype,animal,conflict with vehicles turning left,conflict with vehicles turning right,conflict with vehicles turning righton red,crosswalk markings no longer visible,crosswalk needed,cyclist,drivers don't stop or yield to pedestrians,ice,leaves,...,vehicle turning left (%),vehicle turning right (%),vehicle turning right on red (%),vehicle-from-behind (%),vehicle-turning-head-on (%),vehicle-turning-left (%),vehicle-turning-right (%),vehicles - number makes uncomfortable (%),vehicles - speed makes uncomfortable (%),sum_field
hour,,,,,,,,,,,,,,,,,,,,,
0,0,1,1,0,0,1,0,6,0,0,...,5.8,2.9,0.0,0.0,0.0,0.0,0.0,1.4,8.7,69
1,1,1,2,0,1,2,0,8,1,0,...,6.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,3.6,84
2,0,0,0,0,0,1,0,2,0,0,...,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.2,45
3,1,0,0,0,1,1,0,2,1,0,...,0.0,4.4,0.0,0.0,0.0,0.0,0.0,2.2,6.7,45
4,0,0,0,0,0,2,0,4,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9,11.8,34
5,0,0,0,0,0,2,0,2,1,0,...,0.0,3.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
6,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
7,0,0,0,0,0,0,0,2,0,0,...,0.0,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
8,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [88]:
f"{res_path}/{main_var}.xlsx"

'output/time/hour.xlsx'

type,amenity,hazard-concern,incident
hour,,,
0,0.362319,0.521739,0.115942
1,0.238095,0.511905,0.250000
2,0.288889,0.644444,0.066667
3,0.511111,0.422222,0.066667
4,0.441176,0.529412,0.029412
5,0.444444,0.481481,0.074074
6,0.285714,0.714286,0.000000
7,0.000000,0.727273,0.272727
8,0.500000,0.500000,0.000000
